## Import Libraries

In [7]:
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from flask import Flask, request, jsonify
from flask_cors import CORS

## Load data

In [8]:
df_review_imb = pd.read_csv('dataset.csv')

## Split data into train and test

In [9]:
train, test = train_test_split(df_review_imb, test_size=0.33, random_state=42)
train_x, train_y = train['review'], train['sentiment']
test_x, test_y = test['review'], test['sentiment']

## Vectorize the data

In [10]:
tfidf = TfidfVectorizer(stop_words='english')
train_x_vector = tfidf.fit_transform(train_x)
test_x_vector = tfidf.transform(test_x)

## Train the model

In [11]:
svc = SVC(C=1,kernel='linear')
svc.fit(train_x_vector, train_y)

pickle.dump(svc, open('model.pkl', 'wb'))
pickle.dump(tfidf, open('tfidf.pkl', 'wb'))

SVC(C=1, kernel='linear')

## Setting up the Flask API

In [12]:
app = Flask(__name__)
CORS(app)


@app.route('/predict', methods=['POST'])
def predict():
    review = request.form['review']
    prediction = svc.predict(tfidf.transform([review]))
    response = jsonify({'prediction': prediction[0]})
    response.headers.add('Access-Control-Allow-Origin', '*')
    return response


app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [12/Apr/2023 22:53:18] "POST /predict HTTP/1.1" 200 -
